In [13]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.debugger import DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.debugger_options import DebugOptions
from debugger.strainer import StrainerStatus
from benchmark_consts import *

In [14]:
qs = UNSTABLE_MARIPOSA
options = DebugOptions()

mv = BenchViewer(qs, options)

[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] d4e5e77cbc trace slow unknown, fallback to fast_fail [WARN] [init] 3ce5c5e4dc trace slow unknown, fallback to fast_fail 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 

[WARN] [proof] no proofs available 
[WARN] [init] 2a3b1202bd trace slow unknown, fallback to fast_fail 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] 5b7d18ab3e trace did not fail, fallback to fast_fail 
[WARN] [init] 0b497bc090 trace did not fail, fallback to fast_fail 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proofs available 
[WARN] [init] e998d08b31 trace slow unknown, fallback to fast_fail 
[WARN] [init] 3cead21983 trace slow unknown, fallback to fast_fail 


In [3]:
mv.status.print_status()

| category                |   count | percentage   |
|-------------------------|---------|--------------|
| StrainerStatus.FINISHED |     541 | 99.27 %      |
| StrainerStatus.NO_PROOF |       4 | 0.73 %       |
| total                   |     545 | 100.00 %     |


In [15]:
from utils.analysis_utils import fmt_percent

bench_total = len(mv.status.tally)
bench_finished = len(mv.status[StrainerStatus.FINISHED])
# to_fixable = 0
to_cases = dict()
ff_cases = dict()

def print_break_down(p, q):
    return f"{p}/{q} ({fmt_percent(p, q, 1)})"

for q in mv.status[StrainerStatus.FINISHED]:
    dbg = mv[q]
    if dbg.mode == DbgMode.TIMEOUT:
        to_cases[q] = q in mv.fixable
    elif dbg.mode == DbgMode.FAST_FAIL:
        ff_cases[q] = q in mv.fixable
    else:
        print("Unexpected mode", dbg.mode)
        assert False

to_fixable = sum(to_cases.values())
ff_fixable = sum(ff_cases.values())

print("Test Finished:", print_break_down(bench_finished, bench_total))
print("\tTOs:", print_break_down(len(to_cases), bench_finished))
print("\t\tTO fixable:", print_break_down(to_fixable, len(to_cases)))
print("\tFFs:", print_break_down(len(ff_cases), bench_finished))
print("\t\tFF fixable:", print_break_down(ff_fixable, len(ff_cases)))

print("Test Not Finished:", print_break_down(len(mv.status[StrainerStatus.NOT_CREATED]), bench_total))
print("No Proofs:", fmt_percent(len(mv.status[StrainerStatus.NO_PROOF]), bench_total))
print("* fixable:", print_break_down(to_fixable + ff_fixable, bench_total))

Test Finished: 541/545 (99.3%)
	TOs: 444/541 (82.1%)
		TO fixable: 276/444 (62.2%)
	FFs: 97/541 (17.9%)
		FF fixable: 52/97 (53.6%)
Test Not Finished: 0/545 (0.0%)
No Proofs: 0.73%
* fixable: 328/545 (60.2%)


In [16]:
for q in mv.status[StrainerStatus.NOT_CREATED]:
    print("./src/debugger3.py -m auto -i", mv[q].name_hash, "--create-project")

In [17]:
print("unfixable:", len(mv.unfixable))
unfixable = set([mv[q].name_hash for q in mv.unfixable])
print(len(unfixable & set(DOUBLETON_TARGETS)))

unfixable: 213
114


In [18]:
for q in mv.status[StrainerStatus.NOT_TESTED]:
    print("./src/make_spaghet.py --not-verus -i ", mv[q].strainer.test_dir)

In [8]:
from utils.cluster_utils import get_sync_commands
from utils.system_utils import write_misc_script

for q in mv.status.tally:
    dbg = mv[q]
    dbg.collect_garbage()

cmds = get_sync_commands("g2001", mv.get_sync_dirs())
write_misc_script("send.sh", cmds)

[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_fvbkv--lib-Base-BitsetLemmas.i.dfy.Impl__BitsetLemmas.__default.all__in__set__implies__all1s__uint64.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/bench_unstable/base.z3/fs_vwasm--compiler-linearizer-queries-Compiler.Linearizer-15.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NO_PROOF data/projs/bench_unstable/base.z3/d_komodo--verified-remove.gen.dfyImpl___module.__default.va__lemma__kom__smc__remove__non__addr__success.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_TESTED data/projs/bench_unstable/base.z3/d_lvbkv--Impl-IndirectionTable.i.dfy.Impl__IndirectionTable.IndirectionTable.IndirectionTableToVal.smt2 
[WARN] [dbg] skipped GC on unfinished project StrainerStatus.NOT_TESTED data/projs/bench_unstable/base.z3/d_fvbkv--Impl-JournalistMarshallingModel.i.dfy.Impl__JournalistMarshallingModel.__de

In [12]:
# print(to_cases)
DOUBLETON_TARGETS = []
for q in to_cases:
    if not to_cases[q]:
        DOUBLETON_TARGETS.append(mv[q].name_hash)
print(DOUBLETON_TARGETS)

['0f400a3f00', '1d32c4d524', 'f6db5998ec', '58d93cd4ed', 'c5633470e3', '38d93c5774', '5e2368e7d9', 'f1fdf3be38', 'c8a3060122', '60095b1a59', '24a1fea1cb', '1afa5361f7', 'e93855aadd', 'b1b86518f3', '28f9cbda22', '45d0bd930b', 'be808dd0c3', '93815c9e8c', 'de19cf9fa2', '3e5ee53876', 'd27df8e3bb', '1c986e634b', '1d23e345a5', 'b6f117b674', 'a9102fb49c', '0defc27a3b', '81f604e545', 'a05a8d0e07', '01b00606af', 'ea5023bb77', '488737f5fe', 'a9ba4f3553', '5741fd7355', '926dcc2129', '3d6f2d9f7a', '72139a143b', 'b8ed14e0c5', '203627efb4', '1996525606', 'd75368331e', 'cd6015582a', 'd874a82c3a', '11e4bbf583', '84faa1a2fa', '862cb1068a', '22bc3230fa', '9b9e796545', '3b176b0694', '05c075acd8', '1de7054a37', '3b1c756824', '7c6738ef40', 'c3e7d9e76e', '0ad8073737', '28b317196a', 'a854e6c8ad', '9967bcf803', '2211f3ba76', '1f3cf5998c', '7072b00e75', 'd347f45fa2', 'bf6d57d525', '06a10772cf', '422a6c0817', 'dc83338098', '9bfdf36ae8', '9adbe2e346', '411ef2f9db', '4040d2a20b', 'cca6928c4f', '443cdbaf14', '72a2